In [177]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


In [178]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [179]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [180]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [181]:
# # import requests
# # from datetime import datetime, timedelta

# # def get_tourist_places(location: str, start_date: str, end_date: str) -> dict:
# #     """
# #     Fetches top tourist places for a given location using Google Places API,
# #     includes opening hours and generates a simple travel itinerary.

# #     Args:
# #         location: City or destination name (e.g., "Paris").
# #         start_date: Trip start date in YYYY-MM-DD format.
# #         end_date: Trip end date in YYYY-MM-DD format.

# #     Returns:
# #         Dictionary with status, places, and itinerary.
# #     """

# #     GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY"

# #     # Convert location to lat/lng
# #     geocode_url = (
# #         f"https://maps.googleapis.com/maps/api/geocode/json"
# #         f"?address={location}&key={GOOGLE_API_KEY}"
# #     )
# #     geo_response = requests.get(geocode_url).json()
# #     if not geo_response.get("results"):
# #         return {"status": "error", "error_message": f"Location not found: {location}"}
    
# #     lat = geo_response["results"][0]["geometry"]["location"]["lat"]
# #     lng = geo_response["results"][0]["geometry"]["location"]["lng"]

# #     # Search tourist attractions
# #     places_url = (
# #         f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# #         f"?location={lat},{lng}"
# #         "&radius=5000"
# #         "&type=tourist_attraction"
# #         f"&key={GOOGLE_API_KEY}"
# #     )
# #     places_response = requests.get(places_url).json()
# #     results = places_response.get("results", [])
# #     if not results:
# #         return {"status": "error", "error_message": f"No tourist places found near {location}"}

# #     # Format places including opening hours
# #     formatted_places = []
# #     place_ids = []
# #     for place in results:
# #         formatted_places.append({
# #             "name": place.get("name"),
# #             "rating": place.get("rating"),
# #             "address": place.get("vicinity"),
# #             "user_ratings_total": place.get("user_ratings_total"),
# #             "place_id": place.get("place_id")
# #         })
# #         place_ids.append(place.get("place_id"))

# #     # Sort by rating
# #     formatted_places = sorted(
# #         formatted_places,
# #         key=lambda x: (x["rating"] or 0),
# #         reverse=True
# #     )

# #     # Fetch detailed info for opening hours
# #     for place in formatted_places:
# #         details_url = (
# #             f"https://maps.googleapis.com/maps/api/place/details/json"
# #             f"?place_id={place['place_id']}"
# #             f"&fields=name,opening_hours"
# #             f"&key={GOOGLE_API_KEY}"
# #         )
# #         details_response = requests.get(details_url).json()
# #         opening_hours = details_response.get("result", {}).get("opening_hours", {}).get("weekday_text", [])
# #         place["opening_hours"] = opening_hours

# #     # Generate simple itinerary
# #     start_dt = datetime.strptime(start_date, "%Y-%m-%d")
# #     end_dt = datetime.strptime(end_date, "%Y-%m-%d")
# #     num_days = (end_dt - start_dt).days + 1

# #     itinerary = {}
# #     for i in range(num_days):
# #         day = start_dt + timedelta(days=i)
# #         # Split attractions evenly across days
# #         day_places = formatted_places[i::num_days]  # round-robin distribution
# #         itinerary[day.strftime("%Y-%m-%d")] = [
# #             {"name": p["name"], "opening_hours": p.get("opening_hours")} for p in day_places
# #         ]

# #     # Return structured result
# #     return {
# #         "status": "success",
# #         "location": location,
# #         "travel_dates": {"start": start_date, "end": end_date},
# #         "places": formatted_places,
# #         "itinerary": itinerary
# #     }

# # # Test
# # print("🌍 Tourist Places Agent with Itinerary Created")
# # print(get_tourist_places("Paris", "2025-03-10", "2025-03-12"))

# # import requests
# # from datetime import datetime, timedelta

# # def get_tourist_places_osm(city: str, start_date: str, end_date: str) -> dict:
# #     """
# #     Fetch tourist places from OpenStreetMap using Overpass API and generate itinerary.

# #     Args:
# #         city: City name (e.g., "Paris")
# #         start_date: Trip start date in YYYY-MM-DD
# #         end_date: Trip end date in YYYY-MM-DD

# #     Returns:
# #         Dictionary with status, list of places, and simple itinerary.
# #     """

# #     # Step 1: Get city bounding box from Nominatim (geocoding)
# #     nominatim_url = f"https://nominatim.openstreetmap.org/search?format=json&q={city}"
# #     response = requests.get(nominatim_url, headers={"User-Agent": "tourist-agent"})
# #     city_data = response.json()

# #     if not city_data:
# #         return {"status": "error", "error_message": f"City not found: {city}"}

# #     city_info = city_data[0]
# #     bbox = city_info['boundingbox']  # [south, north, west, east]
# #     south, north, west, east = bbox

# #     # Step 2: Query Overpass API for tourist attractions
# #     overpass_query = f"""
# #     [out:json][timeout:25];
# #     (
# #       node["tourism"="attraction"]({south},{west},{north},{east});
# #       way["tourism"="attraction"]({south},{west},{north},{east});
# #       relation["tourism"="attraction"]({south},{west},{north},{east});
# #     );
# #     out center;
# #     """
# #     overpass_url = "https://overpass-api.de/api/interpreter"
# #     osm_response = requests.post(overpass_url, data=overpass_query)
# #     osm_data = osm_response.json()
# #     elements = osm_data.get("elements", [])

# #     if not elements:
# #         return {"status": "error", "error_message": f"No tourist attractions found in {city}"}

# #     # Step 3: Format places
# #     places = []
# #     for el in elements:
# #         name = el.get("tags", {}).get("name")
# #         if not name:
# #             continue
# #         place = {
# #             "name": name,
            
            
# #             "tourism_type": el.get("tags", {}).get("tourism"),
# #         }
# #         places.append(place)

# #     # Step 4: Generate simple itinerary
# #     start_dt = datetime.strptime(start_date, "%Y-%m-%d")
# #     end_dt = datetime.strptime(end_date, "%Y-%m-%d")
# #     num_days = (end_dt - start_dt).days + 1

# #     itinerary = {}
# #     for i in range(num_days):
# #         day = start_dt + timedelta(days=i)
# #         # Round-robin distribution
# #         day_places = places[i::num_days]
# #         itinerary[day.strftime("%Y-%m-%d")] = day_places

# #     return {
# #         "status": "success",
# #         "city": city,
# #         "travel_dates": {"start": start_date, "end": end_date},
# #         "places": places,
# #         "itinerary": itinerary
# #     }

# # # Test the agent
# # if __name__ == "__main__":
# #     city_name = "Kerala"
# #     start = "2025-03-10"
# #     end = "2025-03-12"

# #     result = get_tourist_places_osm(city_name, start, end)
# #     print("🌍 Tourist Places Agent (OSM + Overpass API) Created")
# #     print(result)


# import requests

# def get_top10_tripadvisor_attractions(city: str, rapidapi_key: str):
#     """
#     Fetches top 10 tourist attractions in a city from TripAdvisor API (via RapidAPI),
#     sorted by rating, and prints name + rating.
#     """

#     # 1️⃣ Step 1: Search for the city to get its TripAdvisor location_id (or geo details)
#     location_url = "https://tripadvisor1.p.rapidapi.com/locations/search"
#     query_params = {
#         "query": city,
#         "limit": "5",
#         "lang": "en_US"
#     }
#     headers = {
#         "X-RapidAPI-Key": "38df62a18fmsh76c5ea622fe5734p1ccaeejsn80eba9db1c04",
#         "X-RapidAPI-Host": "tripadvisor1.p.rapidapi.com"
#     }

#     response = requests.get(location_url, headers=headers, params=query_params)
#     data = response.json()

#     # Validate the response
#     if not data.get("data"):
#         print(f"❌ No location data found for {city}")
#         return

#     # Pick the first location result
#     loc = data["data"][0]
#     location_id = loc.get("result_object", {}).get("location_id")
#     if not location_id:
#         print("❌ Could not find a valid TripAdvisor location_id for the city")
#         return

#     # 2️⃣ Step 2: Use the location_id to fetch attractions / POIs
#     attractions_url = "https://tripadvisor1.p.rapidapi.com/attractions/list"
#     query_params2 = {
#         "location_id": location_id,
#         "lang": "en_US",
#         "currency": "USD",
#         "lunit": "km",
#         "sort": "ranking",       # sort by ranking or by popularity
#         "limit": "30"            # fetch more, then we pick top 10
#     }

#     response2 = requests.get(attractions_url, headers=headers, params=query_params2)
#     attractions_data = response2.json()

#     # Validate
#     if not attractions_data.get("data"):
#         print(f"❌ No attractions found for location_id = {location_id}")
#         return

#     # 3️⃣ Process & sort by rating
#     attractions = attractions_data["data"]
#     # Filter and sort by rating (descending)
#     attractions_sorted = sorted(
#         [a for a in attractions if a.get("rating")],
#         key=lambda x: float(x.get("rating")),
#         reverse=True
#     )

#     # 4️⃣ Print top 10 attractions
#     print(f"🌍 Top-rated attractions in {city}:")
#     for i, attr in enumerate(attractions_sorted[:10]):
#         name = attr.get("name")
#         rating = attr.get("rating")
#         num_reviews = attr.get("num_reviews", "N/A")
#         print(f"{i+1}. {name} — Rating: {rating}, Reviews: {num_reviews}")


# # ------------------------
# # Example usage
# # ------------------------
# RAPIDAPI_KEY = "38df62a18fmsh76c5ea622fe5734p1ccaeejsn80eba9db1c04"
# get_top10_tripadvisor_attractions("Kerala", RAPIDAPI_KEY)


In [188]:
import requests

def get_top_tourist_places(location: str, api_key: str):
    """
    Fetches the top 10 tourist places for a given location using Geoapify Places API.
    Always prints something, even if names are missing.
    """
    # Step 1: Geocode the location
    geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={location}&apiKey={api_key}"
    geocode_response = requests.get(geocode_url).json()
    
    if not geocode_response.get("features"):
        print("No coordinates found for this location.")
        return []
    
    coords = geocode_response["features"][0]["geometry"]["coordinates"]
    lon, lat = coords[0], coords[1]
    
    # Step 2: Query tourist places
    # places_url = (
    #     f"https://api.geoapify.com/v2/places?"
    #     f"categories=tourism.attraction,tourism.sights,tourism.museum&"
    #     f"filter=circle:{lon},{lat},20000&"  # expanded to 20 km radius
    #     f"limit=10&"
    #     f"apiKey={api_key}"
    # )
    places_url = (
        f"https://api.geoapify.com/v2/places?"
        # f"categories=entertainment,natural,leisure&"
        f"categories=tourism.attraction,tourism.sights&"
        # f"filter=place:51a6b8c038d867534059109f292ee9f92940f00101f9010c95780000000000c0020692030942656e67616c757275&"
        f"filter=circle:{lon},{lat},50000&"        
        f"limit=10&"
        f"apiKey={api_key}"
    )

    
# https://api.geoapify.com/v2/places?categories=tourism.attraction,tourism.sights&filter=circle:77.5939973522254,12.9721091,5000&bias=proximity:77.5939973522254,12.9721091&limit=20&apiKey=YOUR_API_KEY

    print(places_url)
    
    places_response = requests.get(places_url).json()
    
    # Step 3: Extract results with fallback
    tourist_places = []
    for feature in places_response.get("features", []):
        props = feature["properties"]
        name = props.get("name")
        if not name:
            # fallback: use category or generic label
            categories = props.get("categories", [])
            name = categories[0] if categories else "Tourist Place"
        address = props.get("formatted", "No address available")
        tourist_places.append({"name": name, "address": address})
    
    # Step 4: Print results
    print("Top 10 Tourist Places:")
    if not tourist_places:
        print("No tourist places found nearby.")
    else:
        for idx, place in enumerate(tourist_places, start=1):
            print(f"{idx}. {place['name']} - {place['address']}")
    
    return tourist_places


# Example usage:
if __name__ == "__main__":
    API_KEY = "8c0a4028ca814fe0b8b20ce971c4703b"
    location = "Goa, India"
    get_top_tourist_places(location, API_KEY)


https://api.geoapify.com/v2/places?categories=tourism.attraction,tourism.sights&filter=circle:74.0855134,15.3004543,50000&limit=10&apiKey=8c0a4028ca814fe0b8b20ce971c4703b
Top 10 Tourist Places:
1. Fort Aguada - Aguada Fort, Fort Aguada Road, Sinquerim - 403515, Goa, India
2. Reis Magos Fort - Reis Magos Fort, Reis Magos Road, Verem, Reis Magos - 403114, Goa, India
3. Cabo de Rama - Cabo de Rama, Path to nice viewpoint, Kuddi -, Goa, India
4. Churches and Convents of Goa - Churches and Convents of Goa, MDR1, Goa Velha, Neura - 403203, Goa, India
5. Corjuem Fort - Corjuem Fort, Bicholim Road, Aldona, Corjuem - 403502, Goa, India
6. Baga Beach - Baga Beach, Baga, Calangute, Bardez, India
7. Fort Aguada - Fort Aguada Road, Sinquerim - 403515, Goa, India
8. Our Lady of the Hill Chapel - Our Lady of the Hill Chapel, Cumbarjua Road, Old Goa, Gaondali - 403108, Goa, India
9. Archiepiscopal Palace - Archiepiscopal Palace, Diwar Ferry Road, Old Goa, Panaji - 403108, Goa, India
10. Old Goa - Old 

In [183]:
# import requests

# def get_top_cities_in_state(state: str, api_key: str, country: str = "India"):
#     """
#     Fetches the top cities in a given state using Geoapify Geocoding API.
    
#     Args:
#         state (str): State name (e.g., "Karnataka").
#         api_key (str): Your Geoapify API key.
#         country (str): Country name (default "India").
    
#     Returns:
#         list: A list of city names.
#     """
#     # Step 1: Geocode the state to get boundary
#     geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={state}, {country}&type=state&apiKey={api_key}"
#     geocode_response = requests.get(geocode_url).json()
    
#     if not geocode_response.get("features"):
#         print("No state found.")
#         return []
    
#     # Get bounding box of the state
#     bbox = geocode_response["features"][0]["bbox"]
#     lon_min, lat_min, lon_max, lat_max = bbox
    
#     # Step 2: Query for cities within the bounding box
#     places_url = (
#         f"https://api.geoapify.com/v2/places?"
#         f"categories=place.city&"
#         f"filter=rect:{lon_min},{lat_min},{lon_max},{lat_max}&"
#         f"limit=10&"
#         f"apiKey={api_key}"
#     )
    
#     places_response = requests.get(places_url).json()
    
#     # Step 3: Extract city names
#     cities = []
#     for feature in places_response.get("features", []):
#         props = feature["properties"]
#         name = props.get("name", "Unknown City")
#         cities.append(name)
    
#     print(f"Top Cities in {state}:")
#     if not cities:
#         print("No cities found.")
#     else:
#         for idx, city in enumerate(cities, start=1):
#             print(f"{idx}. {city}")
    
#     return cities


# # Example usage:
# if __name__ == "__main__":
#     API_KEY = "8c0a4028ca814fe0b8b20ce971c4703b"
#     state = "Kerala"
#     get_top_cities_in_state(state, API_KEY)


Top Cities in Kerala:
No cities found.


In [191]:
import requests

def find_hotels(location: str, checkin_date: str, checkout_date: str, api_key: str):
    """
    Finds available hotels in the given location for the specified dates.
    Uses Geoapify Places API for hotel listings (availability requires booking API integration).
    
    Args:
        location (str): City or location name (e.g., "Bengaluru, India").
        checkin_date (str): Check-in date in YYYY-MM-DD format.
        checkout_date (str): Check-out date in YYYY-MM-DD format.
        api_key (str): Your Geoapify API key.
    
    Returns:
        list: A list of hotels with name and address.
    """
    # Step 1: Geocode the location to get coordinates
    geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={location}&apiKey={api_key}"
    geocode_response = requests.get(geocode_url).json()
    
    if not geocode_response.get("features"):
        print("No coordinates found for this location.")
        return []
    
    coords = geocode_response["features"][0]["geometry"]["coordinates"]
    lon, lat = coords[0], coords[1]
    
    # Step 2: Query hotels near the coordinates
    places_url = (
        f"https://api.geoapify.com/v2/places?"
        f"categories=accommodation.hotel&"
        f"filter=circle:{lon},{lat},10000&"  # 10 km radius
        f"limit=20&"
        f"apiKey={api_key}"
    )
    
    places_response = requests.get(places_url).json()
    
    # Step 3: Extract hotel results
    hotels = []
    for feature in places_response.get("features", []):
        props = feature["properties"]
        name = props.get("name", "Unnamed Hotel")
        address = props.get("formatted", "No address available")
        hotels.append({
            "name": name,
            "address": address,
            "checkin_date": checkin_date,
            "checkout_date": checkout_date
        })
    
    print(f"Hotels available in {location} from {checkin_date} to {checkout_date}:")
    if not hotels:
        print("No hotels found.")
    else:
        for idx, hotel in enumerate(hotels, start=1):
            print(f"{idx}. {hotel['name']} - {hotel['address']}")
    
    return hotels


# Example usage:
if __name__ == "__main__":
    API_KEY = "8c0a4028ca814fe0b8b20ce971c4703b"
    location = "Bellandur, Bengaluru, India"
    checkin = "2025-12-01"
    checkout = "2025-12-05"
    find_hotels(location, checkin, checkout, API_KEY)


Hotels available in Bellandur, Bengaluru, India from 2025-12-01 to 2025-12-05:
1. Conrad Bengaluru - Conrad Bengaluru, 25/3, Kensington Road, Halasuru, Bengaluru - 560008, Karnataka, India
2. Ibis Hotel - Ibis Hotel, 26/1, AMR Tech Park Road, AMR Tech Park, Bengaluru - 560068, Karnataka, India
3. Hotel Empark - Hotel Empark, 2, 1st Main Road, Shanti Nagar, Bengaluru - 560027, Karnataka, India
4. Fortune Select Trinity - Fortune Select Trinity, 134,135,136, 2nd Road, EPIP Zone, Bengaluru - 560066, Karnataka, India
5. Lemon Tree Hotel - Lemon Tree Hotel, 23, 1st A Road, EPIP Zone, Bengaluru - 560066, Karnataka, India
6. Treebo Cartier Hotel - Treebo Cartier Hotel, Foot bridge, Marathahalli, Bengaluru - 560037, Karnataka, India
7. Oberoi Hotel - Oberoi Hotel, 37-39, Mahatma Gandhi Road, Halasuru, Bengaluru - 560001, Karnataka, India
8. Taj MG Road - Taj MG Road, 41/3, Mahatma Gandhi Road, Halasuru, Bengaluru - 560001, Karnataka, India
9. Sarovar Portico - Sarovar Portico, 39/5, Doddanekku

In [2]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

RAPIDAPI_KEY = "38df62a18fmsh76c5ea622fe5734p1ccaeejsn80eba9db1c04"

# TripAdvisor API base endpoint (RapidAPI)
SEARCH_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchLocation"
ATTRACTIONS_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/attraction/searchAttractions"
# https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchRestaurants?locationId=304554

def fetch_location_id(location: str):
    """Fetch TripAdvisor location_id for a given city/location."""
    headers = {
        "x-rapidapi-key": RAPIDAPI_KEY,
        "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
    }
    params = {"query": location}

    response = requests.get(SEARCH_URL, headers=headers, params=params)
    data = response.json()
    print("data in location id",data)

    if "data" not in data or len(data["data"]) == 0:
        return None

    return data["data"][0]["locationId"]
    
def fetch_best_tourist_places(location: str, start_date: str, end_date: str, limit: int = 10):
    """
    Fetches best tourist attractions for a given location,
    then sorts them by rating (descending).
    """

    location_id = fetch_location_id(location)
    print("location_id",location_id)
    if not location_id:
        return {"error": "Unable to fetch location ID. Try another location."}

    headers = {
        "x-rapidapi-key": RAPIDAPI_KEY,
        "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
    }
    params = {
        "locationId": location_id,
        "sort": "ranking",
        "page": "1",
        "subcategory": "all"
    }

    response = requests.get(ATTRACTIONS_URL, headers=headers, params=params)
    data = response.json()

    if "data" not in data:
        return {"error": "No attractions found."}

    attractions = data["data"]
    print("attractions",attractions)

    # Sort by rating (descending)
    attractions_sorted = sorted(
        attractions,
        key=lambda x: float(x.get("rating", 0) or 0),
        reverse=True
    )[:limit]

    results = []
    for a in attractions_sorted:
        results.append({
            "name": a.get("name"),
            "rating": a.get("rating"),
            "num_reviews": a.get("numReviews"),
            "ranking": a.get("ranking"),
            "address": a.get("addressObj", {}).get("addressString"),
            "photo": a.get("photo", {}).get("images", {}).get("medium", {}).get("url")
        })

    return {
        "location": location,
        "start_date": start_date,
        "end_date": end_date,
        "results": results
    }

if __name__ == "__main__":
    location = "Bangalore"
    start_date = "2025-11-24"
    end_date = "2025-11-30"

    data = fetch_best_tourist_places(location, start_date, end_date, limit=10)

    if "results" in data:
        print(f"\nTop 10 Tourist Places in {location} (Sorted by Rating)\n")
        for idx, place in enumerate(data["results"], 1):
            print(f"{idx}. {place['name']}")
            print(f"   Rating: {place['rating']}")
            print(f"   Reviews: {place['num_reviews']}")
            print(f"   Ranking: {place['ranking']}")
            print(f"   Address: {place['address']}")
            print(f"   Photo: {place['photo']}\n")
    else:
        print("Error:", data.get("error"))


Error: No attractions found.


In [5]:
import requests
import os

RAPIDAPI_KEY = "38df62a18fmsh76c5ea622fe5734p1ccaeejsn80eba9db1c04"

# TripAdvisor API endpoints
SEARCH_LOCATION_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/restaurant/searchLocation"
SEARCH_HOTELS_URL = "https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchHotels"

def fetch_location_id(location: str):
    """Fetch TripAdvisor location_id for a given city/location."""
    headers = {
        "x-rapidapi-key": RAPIDAPI_KEY,
        "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
    }
    params = {"query": location}

    response = requests.get(SEARCH_LOCATION_URL, headers=headers, params=params)
    print("response",response)
    data = response.json()

    if "data" not in data or len(data["data"]) == 0:
        return None

    return data["data"][0]["locationId"]

def fetch_top_hotels(location: str, checkin_date: str, checkout_date: str, max_budget_inr: int, limit: int = 10):
    """
    Fetch top hotels for a given location & dates,
    sorted by rating (descending), filtered by budget (INR).
    """

    location_id = fetch_location_id(location)
    print("location_id",location_id)
    if not location_id:
        return {"error": "Unable to fetch location ID. Try another location."}

    headers = {
        "x-rapidapi-key": RAPIDAPI_KEY,
        "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
    }

    params = {
        "geoId": location_id,
        "checkIn": checkin_date,
        "checkOut": checkout_date,
        "pageNumber": "1",
        "adults": "2",
        "currency": "INR"  # Indian Rupees
    }

    response = requests.get(SEARCH_HOTELS_URL, headers=headers, params=params)
    print("response",response)
    data = response.json()
    print("data",data)

    if "data" not in data:
        return {"error": "No hotels found."}

    hotels = data["data"]["data"]

    filtered_hotels = []

    for h in hotels:
        price_display = h.get("priceForDisplay")  # Example: "₹12,345"
        if not price_display:
            continue

        # Clean price string -> Convert to integer
        try:
            price_clean = int(price_display.replace("₹", "").replace(",", "").strip())
        except:
            continue

        # Filter by user's max budget
        if price_clean <= max_budget_inr:
            bubble = h.get("bubbleRating", {})
            filtered_hotels.append({
                "name": h.get("title"),
                "rating": float(bubble.get("rating", 0)),
                "review_count": bubble.get("count"),
                "price": price_display,
                "numeric_price": price_clean,
                "ranking": h.get("rankingDetails", {}).get("ranking"),
                "address": h.get("geoPoint", {}),
                "photo": h.get("cardPhotos", [{}])[0].get("sizes", {}).get("medium", {}).get("url")
            })

    # Sort by rating (descending)
    filtered_sorted = sorted(filtered_hotels, key=lambda x: x["rating"], reverse=True)[:limit]

    return {
        "location": location,
        "checkin": checkin_date,
        "checkout": checkout_date,
        "budget_inr": max_budget_inr,
        "results": filtered_sorted
    }

if __name__ == "__main__":
    location = "bangalore"
    checkin_date = "2025-11-25"
    checkout_date = "2025-11-27"
    max_budget = "20000"

    data = fetch_top_hotels(location, checkin_date, checkout_date, max_budget_inr=max_budget, limit=10)

    if "results" in data:
        print(f"\nTop Hotels in {location} Under ₹{max_budget} (Sorted by Rating)\n")
        for idx, hotel in enumerate(data["results"], 1):
            print(f"{idx}. {hotel['name']}")
            print(f"   Rating: {hotel['rating']}")
            print(f"   Reviews: {hotel['review_count']}")
            print(f"   Price: {hotel['price']}")
            print(f"   Ranking: {hotel['ranking']}")
            print(f"   Address Geo-Point: {hotel['address']}")
            print(f"   Photo: {hotel['photo']}\n")
    else:
        print("Error:", data.get("error"))


response <Response [200]>
location_id 297628
response <Response [200]>
data {'status': True, 'message': 'Success', 'timestamp': 1763998508653, 'data': {'data': [{'id': '14943807', 'title': '1. Radisson Hotel Bengaluru City Center', 'primaryInfo': 'Free breakfast available', 'secondaryInfo': 'Ulsoor', 'badge': {}, 'bubbleRating': {'count': '(2,172)', 'rating': 4.8}, 'isSponsored': False, 'accentedLabel': False, 'provider': 'Radissonhotels.com', 'priceForDisplay': '$191', 'strikethroughPrice': None, 'priceDetails': None, 'priceSummary': 'View all 9 deals from $172', 'cardPhotos': [{'__typename': 'AppPresentation_PhotoItem', 'sizes': {'__typename': 'AppPresentation_PhotoItemSizeDynamic', 'maxHeight': 625, 'maxWidth': 1024, 'urlTemplate': 'https://dynamic-media-cdn.tripadvisor.com/media/photo-o/31/53/c8/d9/caption.jpg?w={width}&h={height}&s=1'}}, {'__typename': 'AppPresentation_PhotoItem', 'sizes': {'__typename': 'AppPresentation_PhotoItemSizeDynamic', 'maxHeight': 1000, 'maxWidth': 1500, 